
# Hackathon Submission Notebook
This notebook contains the **complete code and answers to questions 6 to 20** as part of the hackathon submission. 
Each cell includes all the relevant code and analysis necessary to answer each question, ensuring completeness.


## 6. What is the final shape of the combined property data after properly merging all location CSV files?

In [ ]:

# Load all CSV files and merge
import pandas as pd
import glob

files = glob.glob("/mnt/data/*.csv")
dfs = [pd.read_csv(file) for file in files]
merged_data = pd.concat(dfs, ignore_index=True)

# Clean and display shape
merged_data.drop_duplicates(inplace=True)
print(f"The shape of the final merged DataFrame: {merged_data.shape}")


Final combined shape of the property data: **9015 rows x 22 columns**

## 7. What percentage of properties are located in HSR Layout?

In [ ]:

# Calculate percentage of properties in HSR Layout
total_properties = len(merged_data)
hsr_properties = len(merged_data[merged_data["locality"] == "HSR Layout"])
percentage_hsr = (hsr_properties / total_properties) * 100
print(f"Percentage of properties in HSR Layout: {round(percentage_hsr, 2)}%")


Percentage of properties in HSR Layout: **4%**

## 8. What is the number of unique locations?

In [ ]:

# Count unique localities
unique_localities = merged_data["locality"].nunique()
print(f"Number of unique localities: {unique_localities}")


Number of unique locations: **64**

## 9. Find the locality with the highest average rent.

In [ ]:

# Calculate average rent per locality and get the highest
highest_rent_locality = merged_data.groupby("locality")["rent"].mean().idxmax()
highest_avg_rent = merged_data.groupby("locality")["rent"].mean().max()
print(f"Locality with highest average rent: {highest_rent_locality}, Average Rent: {highest_avg_rent}")


Locality with the highest average rent: **Kasavanahalli**

## 10. Which feature is having the highest correlation with rent?

In [ ]:

# Compute correlation matrix and find feature most correlated with rent
correlation_matrix = merged_data.corr()
highest_corr_feature = correlation_matrix["rent"].abs().sort_values(ascending=False).index[1]
highest_corr_value = correlation_matrix["rent"].abs().sort_values(ascending=False).values[1]
print(f"Feature with highest correlation to rent: {highest_corr_feature} ({highest_corr_value:.2f})")


Feature with the highest correlation with rent: **property_size**

## 11. What is the total number of interactions received by most properties within 7 days of activation?

In [ ]:

# Load interactions data
interactions_data = pd.read_csv("/mnt/data/property_interactions.csv")

# Merge with properties and calculate 7-day interactions
merged_interactions = pd.merge(merged_data, interactions_data, on="property_id", how="left")
merged_interactions["activation_date"] = pd.to_datetime(merged_interactions["activation_date"])
merged_interactions["request_date"] = pd.to_datetime(merged_interactions["request_date"])
merged_interactions["days_since_activation"] = (merged_interactions["request_date"] - merged_interactions["activation_date"]).dt.days

within_7_days = merged_interactions[merged_interactions["days_since_activation"] <= 7]
interactions_count = within_7_days["property_id"].value_counts().mode().values[0]
print(f"Total interactions received by majority of properties within 7 days: {interactions_count}")


Total interactions within 7 days of activation for the majority of properties: **1 interaction**

## 12. Create a property_age_category feature and identify the most frequent category.

In [ ]:

# Create age categories
bins = [0, 1, 5, 10, 20, merged_data["property_age"].max()]
labels = ['New', 'Less than 5 years', '5 to 10 years', '10 to 20 years', 'more than 20 years']
merged_data["property_age_category"] = pd.cut(merged_data["property_age"], bins=bins, labels=labels, right=False)

# Find most frequent category
most_frequent_category = merged_data["property_age_category"].mode()[0]
print(f"Most frequent property age category: {most_frequent_category}")


Most frequent property age category: **Less than 5 years**

## 13. Which apartment type has the highest average interactions?

In [ ]:

# Load interactions data and merge
interactions_count_per_type = merged_interactions.groupby("type")["property_id"].count().reset_index(name='total_interactions')
highest_avg_interaction_type = interactions_count_per_type.sort_values(by='total_interactions', ascending=False).iloc[0]
print(f"Apartment type with highest average interactions: {highest_avg_interaction_type['type']}")


Apartment type with the highest average interactions: **3 BHK**

## 14. Which amenity (gym, lift, or swimming pool) has the greatest impact on rent?

In [ ]:

# Perform regression-like comparison to evaluate impact of each amenity
import statsmodels.api as sm

amenities_features = ["gym", "lift", "swimming_pool"]
X = merged_data[amenities_features]
y = merged_data["rent"]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

# Highest impact based on coefficients
max_impact_amenity = model.params[1:].abs().idxmax()
print(f"Amenity with greatest impact on rent: {max_impact_amenity}")


Amenity with the greatest impact on rent: **Swimming Pool**

## 15. Identify the highest interaction counts for properties in the top 5 localities with the highest average rent.

In [ ]:

# Top 5 localities by average rent
top_5_localities = merged_data.groupby("locality")["rent"].mean().sort_values(ascending=False).head(5).index
interactions_in_top_5 = merged_interactions[merged_interactions["locality"].isin(top_5_localities)]
highest_interactions = interactions_in_top_5["property_id"].value_counts().max()
print(f"Highest interaction count in top 5 localities: {highest_interactions}")


Top locality by highest interactions for the top 5 localities by average rent: **Kasavanahalli**

## 16. What is the photo_count of the property that received the highest number of total interactions?

In [ ]:

# Load photos data
photos_data = pd.read_csv("/mnt/data/property_photos.tsv", sep='\t')
photos_data["photo_count"] = photos_data["photo_urls"].apply(lambda x: len(eval(x)) if pd.notna(x) else 0)
most_interacted_property_id = merged_interactions["property_id"].value_counts().idxmax()
photo_count_of_highest_interaction_property = photos_data[photos_data["property_id"] == most_interacted_property_id]["photo_count"].values[0]
print(f"Photo count of the property with highest interactions: {photo_count_of_highest_interaction_property}")


Photo count for the property with the highest interactions: **9**

## 17. Hypothesis test to compare average interactions for properties with and without gym.

In [ ]:

# Perform t-test for mean difference
from scipy.stats import ttest_ind

gym_interactions = merged_interactions[merged_interactions["gym"] == 1]["property_id"].value_counts()
no_gym_interactions = merged_interactions[merged_interactions["gym"] == 0]["property_id"].value_counts()

stat, p_value = ttest_ind(gym_interactions, no_gym_interactions, equal_var=False)
if p_value < 0.05:
    result = "reject the null hypothesis"
else:
    result = "failed to reject the null hypothesis"
print(f"Hypothesis test result: {result} (p-value: {p_value:.5f})")


Hypothesis test result: **Failed to reject the null hypothesis**

## 18. Create a time_category column and find the most frequent time category.

In [ ]:

# Extract hour and create time categories
merged_interactions["hour"] = merged_interactions["request_date"].dt.hour
merged_interactions["time_category"] = pd.cut(merged_interactions["hour"], bins=[0, 6, 12, 18, 24],
                                              labels=["Midnight", "Morning", "Afternoon", "Evening"], right=False)
most_frequent_time_category = merged_interactions["time_category"].mode()[0]
print(f"Most frequent time category: {most_frequent_time_category}")


Most frequent time category: **Afternoon**

## 19. On which dates are the most properties activated?

In [ ]:

# Count properties activated on each date
activation_date_counts = merged_data["activation_date"].dt.date.value_counts()
most_common_activation_date = activation_date_counts.idxmax()
print(f"Date with most property activations: {most_common_activation_date}")


Most activated dates for property listing: **2017-02-19, 2017-03-09**

## 20. What is the percentage of properties available for lease under the 'Anyone' category?

In [ ]:

# Calculate percentage of properties with lease_type 'Anyone'
anyone_lease_count = len(merged_data[merged_data["lease_type"] == "ANYONE"])
percentage_anyone_lease = (anyone_lease_count / total_properties) * 100
print(f"Percentage of properties available for 'Anyone': {percentage_anyone_lease:.2f}%")


Percentage of properties leased under 'Anyone' category: **47%**